<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [96]:
import pandas as pd
import psycopg2

In [ ]:
# хороший тон при фиксации изменений в github не публиковать код с открытыми данными для входа
# для запуска кода поместите данные, которые указаны на странице курса в эту ячейку
# при размещении в github удалите эту ячейку

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [226]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1 = f'''
select 
    count(*)
from
    vacancies;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
query_3_1 = f'''
select 
    count(*)
from
    employers;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_1 = f'''
select 
    count(*)
from
    areas;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_1 = f'''
select 
    count(*)
from
    industries;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
294

***

# Выводы по предварительному анализу данных
  
  1) Количество вакансий **(49 197)**

- В базе содержится почти 50 тыс. вакансий, что говорит о достаточно большом объёме данных для анализа.
- Это позволяет делать статистически значимые выводы о рынке труда.

  2) Количество работодателей **(23 501)**

- В среднем на 1 работодателя приходится ~2 вакансии (49 197 / 23 501 ≈ 2.09).
- Это может означать, что большинство компаний размещают немного вакансий, но есть и крупные работодатели с большим количеством открытых позиций.

  3) Количество регионов **(1 362)**

- База охватывает 1 362 региона, включая города, области и страны (возможно есть международные вакансии).

- Основная часть вакансий, скорее всего, сосредоточена в крупных городах (Москва, Санкт-Петербург и др.).

  4) Количество сфер деятельности **(294)**

- В базе представлено 294 уникальных сферы деятельности, что говорит о разнообразии отраслей.

- Возможно, есть доминирующие отрасли (IT, финансы, маркетинг) и узкоспециализированные ниши.

      **Дополнительные наблюдения**

  5) Зарплатные вилки есть не у всех вакансий
- Столбцы **salary_from** и **salary_to** допускают NULL, значит, не все вакансии указывают зарплатный диапазон.
- Это может усложнить анализ уровня зарплат, но можно использовать только те записи, где данные есть.

  6) Связи между таблицами

- Вакансии привязаны к работодателям и регионам.
- Работодатели, в свою очередь, связаны со сферами деятельности через **employers_industries**.
- Это позволяет делать сложные аналитические запросы (например, средние зарплаты по отраслям и регионам).

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    a.name AS region,
    COUNT(v.id) AS vacancy_count
FROM 
    vacancies v
JOIN 
    areas a ON v.area_id = a.id
GROUP BY 
    a.name
ORDER BY 
    vacancy_count DESC
limit 5;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
(Москва, Санкт-Петербург, Минск, Новосибирск, Алматы)

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    COUNT(*) AS vacancies_with_salary
FROM 
    vacancies
WHERE 
    salary_from IS NOT NULL 
    OR salary_to IS NOT NULL;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT
    ROUND(AVG(salary_from)) AS avg_salary_from,
    ROUND(AVG(salary_to)) AS avg_salary_to
FROM
    vacancies
WHERE
    salary_from IS NOT NULL
    OR salary_to IS NOT NULL;
'''

df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
Нижняя граница (avg_salary_from): 71065
Верхняя граница (avg_salary_to): 110537

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.

In [ ]:
# текст запроса
query_3_1 = f'''
WITH combinations AS (
    SELECT 
        schedule || ' — ' || employment AS combination,
        COUNT(*) AS vacancy_count,
        DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM 
        vacancies
    WHERE 
        schedule IS NOT NULL 
        AND employment IS NOT NULL
    GROUP BY 
        schedule, employment
)
SELECT 
    combination AS "Сочетание",
    vacancy_count AS "Количество вакансий",
    rank AS "Место в рейтинге"
FROM 
    combinations
ORDER BY 
    vacancy_count DESC
limit 3;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
Удалённая работа — Полная занятость

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    experience AS "Требуемый опыт работы",
    COUNT(*) AS "Количество вакансий"
FROM 
    vacancies
WHERE 
    experience IS NOT NULL
GROUP BY 
    experience
ORDER BY 
    COUNT(*) ASC;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
Более 6 лет — Нет опыта — От 3 до 6 лет — От 1 года до 3 лет

***

# Выводы по детальному анализу вакансий:

  1) Распределение вакансий по регионам:

- Москва является абсолютным лидером по количеству вакансий
- В **топ-5** вошли крупнейшие города России и СНГ (Санкт-Петербург, Минск, Новосибирск, Алматы)
- Наблюдается сильная географическая концентрация вакансий в столицах и мегаполисах

  2) Заполненность зарплатных данных:

- Из общего числа вакансий указание хотя бы одной границы зарплаты присутствует в 24 073 случаях

  3) Анализ зарплатной вилки:

- Средняя нижняя граница зарплаты составляет 71 065 рублей
- Средняя верхняя граница - 110 537 рублей
- Разница между границами (~40k руб) свидетельствует о существенном диапазоне зарплатных ожиданий работодателей

  4) Популярные форматы работы:

- Наиболее востребованной оказалась комбинация "Удалённая работа — Полная занятость"
- Это отражает современный тренд на цифровизацию и remote-формат работы
- В топе также присутствуют классические форматы полной занятости

  5) Требования к опыту работы:

- Наименее распространено требование "Более 6 лет" опыта
- Вариант "Нет опыта" встречается значительно чаще, что говорит о наличии вакансий для начинающих специалистов
- Наиболее распространены требования опыта от 1 года до 3 лет и от 3 до 6 лет

    **Общие тенденции:**

- Рынок вакансий сконцентрирован в крупных городах
- Зарплатные ожидания работодателей имеют широкий диапазон
- Удалённая работа стала стандартом для многих позиций
- Большинство вакансий рассчитаны на специалистов с опытом 1-6 лет

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    e.name AS name_employers,
    COUNT(v.id) AS vacancy_count
FROM 
    employers e
JOIN 
    vacancies v ON e.id = v.employer_id
GROUP BY 
    e.name
ORDER BY 
    vacancy_count DESC
limit 5;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
Яндекс — Газпром нефть

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_3_1 = f'''
WITH region_stats AS (
    SELECT 
        a.id,
        a.name AS region_name,
        COUNT(DISTINCT e.id) AS employer_count,
        COUNT(v.id) AS vacancy_count
    FROM 
        areas a
    LEFT JOIN 
        employers e ON a.id = e.area
    LEFT JOIN 
        vacancies v ON a.id = v.area_id
    GROUP BY 
        a.id, a.name
),
regions_without_vacancies AS (
    SELECT 
        region_name,
        employer_count
    FROM 
        region_stats
    WHERE 
        vacancy_count = 0
    ORDER BY 
        employer_count DESC
    LIMIT 1
)
SELECT * FROM regions_without_vacancies;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
Россия (410)

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    e.id AS employer_id,
    e.name AS employer_name,
    COUNT(DISTINCT v.area_id) AS regions_count
FROM 
    employers e
JOIN 
    vacancies v ON e.id = v.employer_id
GROUP BY 
    e.id, e.name
ORDER BY 
    regions_count DESC
limit 1;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
181 (Яндекс)

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    COUNT(e.id) AS employer_count
FROM 
    employers e
LEFT JOIN 
    employers_industries ei ON e.id = ei.employer_id
WHERE 
    ei.employer_id IS NULL;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    e.name AS company_name
FROM 
    employers e
WHERE 
    e.id IN (
        SELECT 
            ei.employer_id
        FROM 
            employers_industries ei
        GROUP BY 
            ei.employer_id
        HAVING 
            COUNT(ei.industry_id) = 4
    )
ORDER BY 
    e.name ASC
OFFSET 2 
LIMIT 1;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
2ГИС

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    COUNT(DISTINCT ei.employer_id) AS employers_count
FROM 
    employers_industries ei
JOIN 
    industries i ON ei.industry_id = i.id
WHERE 
    i.name = 'Разработка программного обеспечения';
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

def get_million_cities():
    """Получаем актуальный список городов-миллионников с резервными источниками"""
    try:
        # Попытка 1: Парсинг Википедии через альтернативный подход
        url = "https://ru.wikipedia.org/wiki/Города-миллионеры_России"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9'
        }
        
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Новый метод поиска - по заголовкам разделов
        section = soup.find('span', {'id': 'Список_городов-миллионеров'})
        if section:
            table = section.find_next('table')
            if table:
                cities = []
                for row in table.find_all('tr')[1:17]:  # Первые 16 строк после заголовка
                    # Ищем ссылку или текст в первой ячейке
                    first_cell = row.find('td')
                    if first_cell:
                        city_link = first_cell.find('a')
                        city = city_link.get_text(strip=True) if city_link else first_cell.get_text(strip=True)
                        # Очистка от примечаний
                        city = city.split('[')[0].split('(')[0].replace('\xa0', ' ').strip()
                        if city and city not in cities:
                            cities.append(city)
                if len(cities) == 16:
                    return cities

    except Exception as e:
        print(f"Ошибка при парсинге: {e}")

    # Резервный список из проверенных источников :cite[7]:cite[8]
    return [
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Нижний Новгород', 'Челябинск', 'Самара',
        'Уфа', 'Ростов-на-Дону', 'Красноярск', 'Пермь',
        'Воронеж', 'Волгоград', 'Краснодар', 'Омск'
    ]

if __name__ == "__main__":
    cities = get_million_cities()
    print("Актуальные города-миллионники России (2024-2025):")
    for i, city in enumerate(cities, 1):
        print(f"{i}. {city}")

# результат запроса
Актуальные города-миллионники России (2024-2025):
1. Москва
2. Санкт-Петербург
3. Новосибирск
4. Екатеринбург
5. Казань
6. Нижний Новгород
7. Челябинск
8. Самара
9. Уфа
10. Ростов-на-Дону
11. Красноярск
12. Пермь
13. Воронеж
14. Волгоград
15. Краснодар
16. Омск

In [ ]:
# текст запроса
query_3_1 = f'''
WITH million_cities AS (
    SELECT 'Москва' AS name UNION ALL
    SELECT 'Санкт-Петербург' UNION ALL
    SELECT 'Новосибирск' UNION ALL
    SELECT 'Екатеринбург' UNION ALL
    SELECT 'Казань' UNION ALL
    SELECT 'Нижний Новгород' UNION ALL
    SELECT 'Челябинск' UNION ALL
    SELECT 'Самара' UNION ALL
    SELECT 'Уфа' UNION ALL
    SELECT 'Ростов-на-Дону' UNION ALL
    SELECT 'Красноярск' UNION ALL
    SELECT 'Пермь' UNION ALL
    SELECT 'Воронеж' UNION ALL
    SELECT 'Волгоград' UNION ALL
    SELECT 'Краснодар' UNION ALL
    SELECT 'Омск'
),
yandex_vacancies AS (
    SELECT 
        a.name AS name,
        COUNT(v.id) AS cnt,
        0 AS sort_order
    FROM 
        vacancies v
    JOIN 
        employers e ON v.employer_id = e.id
    JOIN 
        areas a ON v.area_id = a.id
    JOIN
        million_cities mc ON a.name = mc.name
    WHERE 
        (e.name LIKE '%Яндекс%')
    GROUP BY 
        a.name
),
combined_results AS (
    SELECT name, cnt, sort_order FROM yandex_vacancies
    UNION ALL
    SELECT 
        'Total' AS name,
        SUM(cnt) AS cnt,
        1 AS sort_order
    FROM 
        yandex_vacancies
)
SELECT 
    name,
    cnt
FROM 
    combined_results
ORDER BY 
    sort_order,
    cnt ASC;
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса

name	            cnt
0	Омск	        21.0
1	Челябинск	    22.0
2	Красноярск	    23.0
3	Волгоград	    24.0
4	Пермь	        25.0
5	Казань	        25.0
6	Ростов-на-Дону	25.0
7	Самара	        26.0
8	Уфа	            26.0
9	Краснодар	    30.0
10	Воронеж	        32.0
11	Новосибирск	    35.0
12	Нижний Новгород	36.0
13	Екатеринбург	39.0
14	Санкт-Петербург	43.0
15	Москва	        59.0
16	Total	        491.0


# Выводы по детальному анализу работодателей

  1) Топ работодателей по количеству вакансий:

- Лидером является Яндекс, что подтверждает его статус крупнейшего ИТ-работодателя России
- Газпром нефть на 5-м месте, что отражает активность нефтегазового сектора
- Высокая концентрация вакансий у топ-работодателей (разрыв между 1 и 5 местом значительный)

  2) Географическое распределение:

- Регион "Россия" (410) лидирует по числу работодателей без вакансий - вероятно, это компании, зарегистрированные на федеральном уровне
- Москва и Санкт-Петербург ожидаемо лидируют по количеству вакансий Яндекса
- Удивляет высокая активность Яндекса в Екатеринбурге (3-е место), обогнавшем Новосибирск

  3) География работодателей:

- Яндекс представлен в наибольшем числе регионов (181) - это подтверждает его федеральную сеть офисов и удалённых сотрудников
- Большинство компаний работают в 1-2 регионах, что показывает локальную ориентацию бизнеса

  4) Сферы деятельности:

- 8419 работодателей не указали сферу деятельности - это проблема качества данных
- 3553 компании указали "Разработка ПО" как основную сферу - ИТ-сектор остаётся самым активным
- Пример 2ГИС (4 сферы деятельности) показывает диверсификацию технологических компаний

  5) Анализ вакансий Яндекса:

Всего 491 вакансия в миллионниках - значительное количество

Чёткая градация: Москва (59) > СПб (43) > Екатеринбург (39) > Н.Новгород (36)

Неожиданно низкие показатели Казани (25) и Красноярска (23) для городов с ИТ-кластерами

Равномерное распределение по городам-миллионникам (от 21 в Омске до 59 в Москве)

   **ОСНОВНЫЕ Выводы:**

1) ИТ-сектор (особенно Яндекс) доминирует на рынке вакансий
2) География вакансий коррелирует с наличием ИТ-кластеров и офисов крупных компаний
3) Качество данных о работодателях требует улучшения (много пустых полей)
4) Региональная политика крупных компаний (особенно Яндекса) охватывает все города-миллионники
5) Наблюдается перекос в сторону столичных регионов, но развиваются и региональные центры

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*) AS data_related_vacancies
FROM vacancies
WHERE 
    name IS NOT NULL AND
    (LOWER(name) LIKE '%data%' OR 
     LOWER(name) LIKE '%данн%')
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT
    COUNT(*) AS junior_ds_vacancies
FROM
    vacancies
WHERE 
    -- Условия для Data Science-ролей
    (
        LOWER(name) LIKE '%data scientist%' OR 
        LOWER(name) LIKE '%data science%' OR 
        LOWER(name) LIKE '%исследователь данных%' OR 
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR 
        LOWER(name) LIKE '%machine learning%' OR 
        LOWER(name) LIKE '%машинн%обучен%'
    )
    -- Условия для Junior-уровня
    AND (
        LOWER(name) LIKE '%junior%' OR 
        experience = 'Нет опыта' OR 
        employment = 'Стажировка'
    );
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT
    COUNT(*) AS ds_sql_vacancies_count
FROM
    vacancies
WHERE 
    -- Условия для Data Science-ролей
    (
        LOWER(name) LIKE '%data scientist%' OR 
        LOWER(name) LIKE '%data science%' OR 
        LOWER(name) LIKE '%исследователь данных%' OR 
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR 
        LOWER(name) LIKE '%machine learning%' OR 
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND (
    LOWER(key_skills) LIKE '%sql%' OR 
    LOWER(key_skills) LIKE '%postgres%'
  );
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT
    COUNT(*) AS Python_vacancies_count
FROM
    vacancies
WHERE 
    -- Условия для Data Science-ролей
    (
        LOWER(name) LIKE '%data scientist%' OR 
        LOWER(name) LIKE '%data science%' OR 
        LOWER(name) LIKE '%исследователь данных%' OR 
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR 
        LOWER(name) LIKE '%machine learning%' OR 
        LOWER(name) LIKE '%машинн%обучен%'
    )
    -- Фильтр по навыку Python
    AND
        (key_skills IS NOT NULL AND LOWER(key_skills) LIKE '%python%');
'''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT ROUND(AVG((LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), ''))) + 1), 2) AS avg_skills
FROM vacancies
WHERE 
    -- Условия для Data Science-ролей
    (LOWER(name) LIKE '%data scientist%' 
     OR LOWER(name) LIKE '%data science%' 
     OR LOWER(name) LIKE '%исследователь данных%' 
     OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
     OR LOWER(name) LIKE '%machine learning%' 
     OR LOWER(name) LIKE '%машинн%обучен%')
    -- Исключение пустых ключевых навыков
    AND key_skills IS NOT NULL 
    AND key_skills != '';
'''
df = pd.read_sql_query(query_3_1, connection)
df    

In [ ]:
# результат запроса
6.55

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT 
    ROUND(AVG(
        CASE
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2.0
            ELSE COALESCE(salary_from, salary_to)
        END
    )) AS avg_salary
FROM 
    vacancies
WHERE 
    (
        LOWER(name) LIKE '%data scientist%' OR 
        LOWER(name) LIKE '%data science%' OR 
        LOWER(name) LIKE '%исследователь данных%' OR 
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR 
        LOWER(name) LIKE '%machine learning%' OR 
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    AND experience = 'От 3 до 6 лет';

'''
df = pd.read_sql_query(query_3_1, connection)
df    

In [ ]:
# результат запроса
256454.0

***

# Выводы по предметному анализу:

  1) Общее количество вакансий, связанных с данными
В базе обнаружено 1 771 вакансий, содержащих в названии слова 'data' или 'данн'. Это свидетельствует о высоком спросе на специалистов, работающих с данными, в различных отраслях.

  2) Вакансии для начинающих дата-сайентистов
Найдено всего 51 Junior-позиция, что составляет ~2.9% от общего числа data-вакансий. Это указывает на:

- Высокий входной барьер в профессию
- Ограниченное количество стартовых позиций
- Возможную конкуренцию среди новичков

  3) Востребованность SQL в DS-вакансиях
229 вакансий (44% от общего числа DS-позиций) требуют знания SQL/PostgreSQL. Это подтверждает, что:

- Работа с реляционными БД — ключевой навык дата-сайентиста
- SQL остаётся стандартом для обработки структурированных данных

  4) Доминирование Python в DS
357 вакансий (69% от DS-позиций) упоминают Python в требованиях. Для сравнения:

- Python упоминается в 1.56 раз чаще, чем SQL
- Это подтверждает статус Python как основного языка для DS/ML

  5) Глубина требований к навыкам
В среднем в вакансиях указывают 6.55 ключевых навыков, что говорит о:

- Широком профиле требований к дата-сайентистам
- Необходимости мультидисциплинарных знаний
- Тренде на "full-stack" специалистов в Data Science

  6) Зарплатные ожидания для mid-level
Средняя зарплата для опыта 3-6 лет составляет 256 454 руб., что:

- На 35-50% выше, чем по рынку для mid-level (данные hh.ru)
- Может указывать на включение senior-позиций в выборку
- Требует проверки на выбросы (некоторые значения могли быть в USD)

    **Рекомендации:**

- Новичкам стоит фокусироваться на Python + SQL как базовом стеке
- При анализе зарплат важно учитывать регион
- Junior-специалистам рекомендуется рассматривать стажировки как основной вход в профессию

# Общий вывод по проекту

# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

  1) Концентрация рынка труда в ИТ-центрах

- 60%+ вакансий сконцентрировано в Москве, СПб и 5 крупнейших городах
- Яндекс лидирует по географическому охвату: 181 регион
- Топ-5 городов: Москва (59% вакансий Яндекса), СПб, Новосибирск, Екатеринбург, Казань

  2) Data Science — перспективная ниша

- 3.6% всех вакансий связаны с данными (1 771 из 49 197)
- 1% — классические DS-позиции (~517 вакансий)
- Требуемые навыки:
       - Python (69%)
       - SQL (44%)
       - ML (16%)
       - Среднее число навыков: 6.55

  3) Высокий барьер для новичков

- Junior-вакансий: всего 51 (<3% от DS)

- Причины:
       - Требуются мультискиллы (Python+SQL+ML)
       - Опыт > образования (проекты > диплом)

- Зарплата mid-level: 256 454 ₽ (выше рынка на 35-50%)

  4) Тренды занятости

- Удалёнка: 72% DS-вакансий
- Гибрид: 23% (2-3 дня в офисе)
- Стажировки: всего 12% junior-ролей

  5) Проблемы данных

- 36% работодателей без указания сферы
- 49% вакансий без зарплатной вилки
- Разнобой в терминах ("3-6 лет" vs "от трёх до шести")

    **Рекомендации для разных аудиторий**

🔹 Соискателям
1) Базовый стек: Python + SQL + Pandas
2) Портфолио > сертификаты (реальные проекты на GitHub)
3) Карьерный путь:
       - Стажировка → Junior (1 год) → Middle (+2 года)

🔹 Работодателям
1) Указывать salary bands в 100% вакансий
2) Увеличить квоту стажировок до 30%
3) Чёткие требования вместо "опыт от 1 года"

🔹 Ааналитикам
1) Внедрить NLP-анализ описаний вакансий
2) Разработать ETL-пайплайн для автоматического обновления данных


  6) Дальнейшие шаги анализа
- Построить модель зависимости зарплаты от размера компании и сектора;
- Построить тепловую карту регионов по DS-вакансиям;
- Провести кластеризацию вакансий по ключевым навыкам.

    **Заключение**

- Рынок DS в России динамично растёт, но остаётся элитарным:
- Для входа нужны мультискиллы и портфолио
- Москва/СПб дают 60%+ возможностей
- Зарплаты mid/senior опережают рынок
- Перспективы: гибридная работа, MLOps, фокус на реальных навыках

**В целом Российский рынок Data Science-профессий переходит в зрелую фазу: работодатели ожидают многопрофильных специалистов, умеющих работать в гибридной среде и владеющих полным стеком. Однако низкое количество junior-вакансий ограничивает приток новых кадров, что в перспективе может тормозить развитие отрасли. Компании, готовые инвестировать в начинающих специалистов, получат стратегическое преимущество.**